# Feature Engineering

A Python class designed for feature engineering that streamlines the process of generating **aggregate features**, extracting useful time-based or transactional attributes, encoding categorical variables, addressing missing values through imputation or removal, and normalizing/standardizing numerical features to ensure consistency. Additionally, it includes functionality for advanced feature transformations like **creating interaction terms or handling outliers**, all aimed at enhancing model performance and ensuring clean, high-quality data.

In [26]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os
import sys
from importlib import reload

sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [27]:
df = pd.read_csv('../data/data_eda.csv')

In [28]:
df

,Unnamed: 0,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0
1,1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0
2,2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0
3,3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0
4,4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95470,95657,TransactionId_89881,BatchId_96668,AccountId_4841,SubscriptionId_3829,CustomerId_3078,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-1000.0,1000,2019-02-13 09:54:09+00:00,2,0
95471,95658,TransactionId_91597,BatchId_3503,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2019-02-13 09:54:25+00:00,2,0
95472,95659,TransactionId_82501,BatchId_118602,AccountId_4841,SubscriptionId_3829,CustomerId_3874,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2019-02-13 09:54:35+00:00,2,0
95473,95660,TransactionId_136354,BatchId_70924,AccountId_1346,SubscriptionId_652,CustomerId_1709,UGX,256,ProviderId_6,ProductId_19,tv,ChannelId_3,3000.0,3000,2019-02-13 10:01:10+00:00,2,0


In [29]:
# Import the python class
from feature_engineering import DataPreprocessor

# Instantiate the FeatureEngineering class
feature_engineer = DataPreprocessor()

In [30]:
if __name__ == '__main__':
    # Make a copy of the dataframe and reset the index
    df_copy = df.copy().reset_index()
    
    # Identify columns to exclude and categorical columns to encode
    cols_to_drop = ['ProductId', 'BatchId', 'AccountId', 'ProviderId', 'SubscriptionId', 
                    'Value', 'CountryCode', 'CurrencyCode']
    cat_features = ['ProductCategory', 'ChannelId']

    # Drop the identified columns
    df_copy.drop(columns=cols_to_drop, inplace=True)

    # Initialize the DataPreprocessor class
    data_preprocessor = DataPreprocessor()

    # Compute aggregate features
    df_with_agg_features = data_preprocessor.compute_aggregates(df_copy)

    # Add transactional metrics
    df_with_transaction_features = data_preprocessor.add_transactional_metrics(df_with_agg_features)

    # Generate time features
    df_with_time_features = data_preprocessor.generate_time_features(df_with_transaction_features)

    # Transform categorical features
    df_encoded = data_preprocessor.transform_categorical(df_with_time_features, cat_features)

    # Fill missing values
    df_cleaned = data_preprocessor.fill_missing_data(df_encoded)

    # Identify numerical columns to scale, excluding specified columns like 'Amount' and 'FraudResult'
    numeric_cols = df_cleaned.select_dtypes(include='number').columns.tolist()
    exclude_cols = ['Amount', 'FraudResult']  # Add any other columns you wish to exclude from scaling
    numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

    # Scale numerical features
    df_scaled = data_preprocessor.scale_numerical_features(df_cleaned, numeric_cols, mode='minmax')


#### **Create Aggregate Features**
**Total Transaction Amount:** Sum of all transaction amounts for each customer.

**Average Transaction Amount:** Average transaction amount per customer.

**Transaction Count:** Number of transactions per customer.

**Standard Deviation of Transaction Amounts:** Variability of transaction amounts per customer.

In [38]:
df_with_agg_features.head()

,index,Unnamed: 0,TransactionId,CustomerId,ProductCategory,ChannelId,Amount,TransactionStartTime,PricingStrategy,FraudResult,Total_Transactions,Average_Transaction,Transaction_Volume,StdDev_Transaction
0,0,0,TransactionId_76871,CustomerId_4406,airtime,ChannelId_3,1000.0,2018-11-15 02:18:49+00:00,2,0,109921.75,923.712185,119,3042.294251
1,1,1,TransactionId_73770,CustomerId_4406,financial_services,ChannelId_2,-20.0,2018-11-15 02:19:08+00:00,2,0,109921.75,923.712185,119,3042.294251
2,2,2,TransactionId_26203,CustomerId_4683,airtime,ChannelId_3,500.0,2018-11-15 02:44:21+00:00,2,0,1000.00,500.000000,2,0.000000
3,3,3,TransactionId_380,CustomerId_988,utility_bill,ChannelId_3,20000.0,2018-11-15 03:32:55+00:00,2,0,228727.20,6019.136842,38,17169.241610
4,4,4,TransactionId_28195,CustomerId_988,financial_services,ChannelId_2,-644.0,2018-11-15 03:34:21+00:00,2,0,228727.20,6019.136842,38,17169.241610


#### **Normalize/Standardize Numerical Features**

Normalization and standardization are scaling techniques used to bring all numerical features onto a similar scale.

- Normalization: Scales the data to a range of [0, 1].

- Standardization: Scales the data to have a mean of 0 and a standard deviation of 1.

In [32]:
# Display the results
df_scaled

,index,Unnamed: 0,TransactionId,CustomerId,Amount,TransactionStartTime,PricingStrategy,FraudResult,Total_Transactions,Average_Transaction,...,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,0.000000,0.000000,TransactionId_76871,CustomerId_4406,1000.0,2018-11-15 02:18:49+00:00,0.5,0.0,0.557522,0.047184,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000010,0.000010,TransactionId_73770,CustomerId_4406,-20.0,2018-11-15 02:19:08+00:00,0.5,0.0,0.557522,0.047184,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.000021,0.000021,TransactionId_26203,CustomerId_4683,500.0,2018-11-15 02:44:21+00:00,0.5,0.0,0.556944,0.047137,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.000031,0.000031,TransactionId_380,CustomerId_988,20000.0,2018-11-15 03:32:55+00:00,0.5,0.0,0.558153,0.047749,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.000042,0.000042,TransactionId_28195,CustomerId_988,-644.0,2018-11-15 03:34:21+00:00,0.5,0.0,0.558153,0.047749,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95470,0.999958,0.999958,TransactionId_89881,CustomerId_3078,-1000.0,2019-02-13 09:54:09+00:00,0.5,0.0,0.569883,0.047553,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
95471,0.999969,0.999969,TransactionId_91597,CustomerId_3874,1000.0,2019-02-13 09:54:25+00:00,0.5,0.0,0.557249,0.047233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
95472,0.999979,0.999979,TransactionId_82501,CustomerId_3874,-20.0,2019-02-13 09:54:35+00:00,0.5,0.0,0.557249,0.047233,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
95473,0.999990,0.999990,TransactionId_136354,CustomerId_1709,3000.0,2019-02-13 10:01:10+00:00,0.5,0.0,0.561401,0.047261,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


### **Extract Features**
**Transaction Hour:** The hour of the day when the transaction occurred.

**Transaction Day:** The day of the month when the transaction occurred.

**Transaction Month:** The month when the transaction occurred.

**Transaction Year:** The year when the transaction occurred.

In [36]:
df_with_time_features

,index,Unnamed: 0,TransactionId,CustomerId,ProductCategory,ChannelId,Amount,TransactionStartTime,PricingStrategy,FraudResult,...,Transaction_Volume,StdDev_Transaction,Net_Amount,Total_Debits,Total_Credits,Debit_to_Credit_Ratio,Hour_of_Transaction,Day_of_Transaction,Month_of_Transaction,Year_of_Transaction
0,0,0,TransactionId_76871,CustomerId_4406,airtime,ChannelId_3,1000.0,2018-11-15 02:18:49+00:00,2,0,...,119,3042.294251,109921.75,66,53,1.222222,2,15,11,2018
1,1,1,TransactionId_73770,CustomerId_4406,financial_services,ChannelId_2,-20.0,2018-11-15 02:19:08+00:00,2,0,...,119,3042.294251,109921.75,66,53,1.222222,2,15,11,2018
2,2,2,TransactionId_26203,CustomerId_4683,airtime,ChannelId_3,500.0,2018-11-15 02:44:21+00:00,2,0,...,2,0.000000,1000.00,2,0,2.000000,2,15,11,2018
3,3,3,TransactionId_380,CustomerId_988,utility_bill,ChannelId_3,20000.0,2018-11-15 03:32:55+00:00,2,0,...,38,17169.241610,228727.20,26,12,2.000000,3,15,11,2018
4,4,4,TransactionId_28195,CustomerId_988,financial_services,ChannelId_2,-644.0,2018-11-15 03:34:21+00:00,2,0,...,38,17169.241610,228727.20,26,12,2.000000,3,15,11,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95470,95470,95657,TransactionId_89881,CustomerId_3078,financial_services,ChannelId_2,-1000.0,2019-02-13 09:54:09+00:00,2,0,...,573,22554.029939,2438140.00,310,263,1.174242,9,13,2,2019
95471,95471,95658,TransactionId_91597,CustomerId_3874,airtime,ChannelId_3,1000.0,2019-02-13 09:54:25+00:00,2,0,...,43,2274.756582,58499.60,27,16,1.588235,9,13,2,2019
95472,95472,95659,TransactionId_82501,CustomerId_3874,financial_services,ChannelId_2,-20.0,2019-02-13 09:54:35+00:00,2,0,...,43,2274.756582,58499.60,27,16,1.588235,9,13,2,2019
95473,95473,95660,TransactionId_136354,CustomerId_1709,tv,ChannelId_3,3000.0,2019-02-13 10:01:10+00:00,2,0,...,520,3198.051000,840485.00,346,174,1.977143,10,13,2,2019


#### **Encode Categorical Variables**

Convert categorical variables into numerical format by using:

- One-Hot Encoding: Converts categorical values into binary vectors. The following festures are encoded in our data:'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId' and 'PricingStrategy'

In [35]:
df_encoded

,index,Unnamed: 0,TransactionId,CustomerId,Amount,TransactionStartTime,PricingStrategy,FraudResult,Total_Transactions,Average_Transaction,...,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,0.000000,0.000000,TransactionId_76871,CustomerId_4406,1000.0,2018-11-15 02:18:49+00:00,0.5,0.0,0.557522,0.047184,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.000010,0.000010,TransactionId_73770,CustomerId_4406,-20.0,2018-11-15 02:19:08+00:00,0.5,0.0,0.557522,0.047184,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.000021,0.000021,TransactionId_26203,CustomerId_4683,500.0,2018-11-15 02:44:21+00:00,0.5,0.0,0.556944,0.047137,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.000031,0.000031,TransactionId_380,CustomerId_988,20000.0,2018-11-15 03:32:55+00:00,0.5,0.0,0.558153,0.047749,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,0.000042,0.000042,TransactionId_28195,CustomerId_988,-644.0,2018-11-15 03:34:21+00:00,0.5,0.0,0.558153,0.047749,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95470,0.999958,0.999958,TransactionId_89881,CustomerId_3078,-1000.0,2019-02-13 09:54:09+00:00,0.5,0.0,0.569883,0.047553,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
95471,0.999969,0.999969,TransactionId_91597,CustomerId_3874,1000.0,2019-02-13 09:54:25+00:00,0.5,0.0,0.557249,0.047233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
95472,0.999979,0.999979,TransactionId_82501,CustomerId_3874,-20.0,2019-02-13 09:54:35+00:00,0.5,0.0,0.557249,0.047233,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
95473,0.999990,0.999990,TransactionId_136354,CustomerId_1709,3000.0,2019-02-13 10:01:10+00:00,0.5,0.0,0.561401,0.047261,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [34]:
df_scaled.columns

Index(['index', 'Unnamed: 0', 'TransactionId', 'CustomerId', 'Amount',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult',
       'Total_Transactions', 'Average_Transaction', 'Transaction_Volume',
       'StdDev_Transaction', 'Net_Amount', 'Total_Debits', 'Total_Credits',
       'Debit_to_Credit_Ratio', 'Hour_of_Transaction', 'Day_of_Transaction',
       'Month_of_Transaction', 'Year_of_Transaction',
       'ProductCategory_data_bundles', 'ProductCategory_financial_services',
       'ProductCategory_movies', 'ProductCategory_other',
       'ProductCategory_ticket', 'ProductCategory_transport',
       'ProductCategory_tv', 'ProductCategory_utility_bill',
       'ChannelId_ChannelId_2', 'ChannelId_ChannelId_3',
       'ChannelId_ChannelId_5'],
      dtype='object')

In [39]:
# Save extracted and cleaned features to csv
df_scaled.to_csv('../data/extracted_features.csv')